## In this notebook we will explore the data set and look for as many insights as we can get

## Importing the required libraries for analysis and visualisation 


In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import datetime as dt
from sklearn.feature_extraction.text import TfidfVectorizer
from yellowbrick.text import TSNEVisualizer
from yellowbrick.datasets import load_hobbies
from yellowbrick.text import FreqDistVisualizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
nltk.download('all')

## loading the data

In [ ]:
data=pd.read_csv('../input/reddit-india-flair-detection/datafinal.csv')

looking for the data shape 

In [ ]:
data.shape

names of columns

In [ ]:
data.columns

It's clearly obvious unnamed: 0 and id have no use in the data so dropping them 


In [ ]:
data.drop('id',axis=1,inplace=True)

In [ ]:
data.drop('Unnamed: 0', axis=1, inplace= True )

In [ ]:
data.shape

obtaining the first 5 rows of data

In [ ]:
data.head()

In [ ]:
data.info()

### by this we find that body has a lot of null values. This shows that for most of the posts only the title suffice, there is no need of body. 
### plots can be made for the posts having body and no of words in such posts this would give more insights to the data. 
### second in the list which has missing values is comments. This is quite resonable as not all the posts get comments, next thing which can be checked is what is the score of the post with comments and what flair are the posts which get maximum comments. 

#### graphs which we can plot to get more insights of the data 
#### 1. plots of different feature values and which feature value has missing values.
#### 2. NO of posts and the flair they belong that have body.
#### 3. No of words in the body corresponding to the flairs- this will help us determine is body a good parameter or not for model.
#### 4. score of the posts with comments
#### 5. flair which gets maximum comments - this will help us in model 
#### 6. what are thw words or persons mentioned maximum times in comments
#### 7, what is the frequency of posts and comments.
#### 8. what are the embeddings of different words in body, title, comments and combined features

one more insight which we can do is 

In [ ]:
data['flair'].value_counts()

This tells us two thing:-
###### First the data is mostly balanced 
###### second [R]eddiquette has minimum no of posts this is may be because not most of the people post in ediquette section or things concerning this

In [ ]:
# converting to data frame
df=pd.DataFrame(data)

In [ ]:
y=data.columns

In [ ]:
x=data.count()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
features = y 
values = x
ax.bar(y,x)
plt.xticks(rotation=90)
plt.savefig('value.png', dpi=300, bbox_inches='tight')
plt.show()

## getting no of words in title

In [ ]:
temp = df['title'].str.len()
temp.hist(bins = np.arange(0,200,1))
plt.title("no of words in TITLE")
plt.xlabel("length of Title (in words)", fontsize=12)
plt.ylabel("Number of posts", fontsize=12)
plt.savefig('title.png', dpi=300, bbox_inches='tight')
plt.show()

this shows title is an important parameter and would affect our model 

## getting no of words in body

In [ ]:

temp = df['body'].str.len()
temp.hist(bins = np.arange(0,1000,10))
plt.title("no of words in BODY")
plt.xlabel("length of BODY (in words)")
plt.ylabel("Number of posts")
plt.savefig('lenth of words in body.png', dpi=300, bbox_inches='tight')
plt.show()

this shows body is not an important parameter 

## visualising how the flairs fare according to the comments no and score
###### score is calculated by subtracting no of downvotes fro no of upvotes

In [ ]:
fig,ax = plt.subplots()
ax.grid()
df.groupby('flair').plot(x='comms_num', y='score', ax=ax, legend=False)
plt.savefig('comments and upvotes corresponding to the flairs', dpi=300, bbox_inches='tight')
plt.show()

flairs vs upvotes

In [ ]:
df2 = df.groupby("flair").mean()[['score']]

df2.plot(kind='bar', legend=False, grid=True)
plt.title("Average score per flair")

plt.xlabel("Flair")
plt.ylabel("Average score per post")
plt.savefig('score per flair', dpi=300, bbox_inches='tight')
plt.show()


this shows food gets a lot of upvotes probably because we indians are foodies XD.
sports and science and technology get a lot of upvotes, same as poilitics this shows indians are least interested in major topics like corona virus, policy economy

comments per flair

In [ ]:
df3 = df.groupby("flair").mean()[['comms_num']]

df3.plot(kind='bar', legend=False, grid=True)
plt.title("Average no of coments per flair")

plt.xlabel("Flair")
plt.ylabel("Average no of comments per post")
plt.savefig('comms per flair', dpi=300, bbox_inches='tight')
plt.show()


this shows that Indians have a scientific temperament and also nterested in food, sports and politics 

## Let's look at how people post with time what is the frequency of differnt flairs

In [ ]:
df4=df.set_index('timestamp')

no of comments per post during the time stamp

In [ ]:
sns.set(rc={'figure.figsize':(20, 8)})
df4['comms_num'].plot(linewidth=1);
plt.title("Number of comments per post during the timestamp")
plt.xlabel("Timestamp")
plt.ylabel("Number of comments")
plt.savefig('no of comments per time stamp.png')
plt.show()

score per post during the time stamp

In [ ]:
sns.set(rc={'figure.figsize':(20, 8)})
df4['score'].plot(linewidth=1);
plt.title("score per post during the timestamp")
plt.xlabel("Timestamp")
plt.ylabel("score")
plt.savefig('score per post during time stamp.png')
plt.show()

In [ ]:
df1=df.set_index('timestamp')

In [ ]:
df2= df1.groupby('flair')[['comms_num']]
df2.head()

Number of comments per post during the timestamp corresponding to the flairs

In [ ]:
sns.set(rc={'figure.figsize':(20, 8)})
df1.groupby('flair')[['comms_num']].plot(linewidth=1);
plt.title("Number of comments per post during the timestamp corresponding to the flairs")
plt.xlabel("Timestamp")
plt.ylabel("Number of comments")
plt.savefig('no of comments per time stamp corresponding to the flairs.png')
plt.show()

## Let's see different word embeddings and how the word from the corpus cluster

In [ ]:
z=df['flair'].unique()

In [ ]:
df['body'].fillna(" ",inplace=True)


In [ ]:

tfidf = TfidfVectorizer()
x = tfidf.fit_transform(df['body'])
y = df['flair']
tsne = TSNEVisualizer(labels=z)
tsne.fit(x, y)
tsne.poof()

By this we can see words in body don't cluster much together according to the flair but politics is clustered around probably because mentioning bjp, congress and other parties and personnel

for comments



In [ ]:
df['comments'].fillna(" ",inplace=True)

In [ ]:

tfidf = TfidfVectorizer()
x = tfidf.fit_transform(df['comments'])
y = df['flair']
tsne = TSNEVisualizer(labels=z)
tsne.fit(x, y)
tsne.poof()

comments are scattered all around the vector space

In [ ]:

tfidf = TfidfVectorizer()
x = tfidf.fit_transform(df['title'])
y = df['flair']
tsne = TSNEVisualizer(labels=z)
tsne.fit(x, y)
tsne.poof()

titles are clustered according to the flair thus title is an important parameter

combined features

In [ ]:

df['combined_features'].fillna(" ",inplace=True)

In [ ]:

tfidf = TfidfVectorizer()
x = tfidf.fit_transform(df['combined_features'])
y = df['flair']
tsne = TSNEVisualizer(labels=z)
tsne.fit(x, y)
tsne.poof()

by looking at all the features they are also scattered yet large clusters can be formed this shows combined features work fine after title thus can be used for model

## now let's look at the last part of our analysis what is the frequency of different words in the comments, body, title etc and which is the word used the most

In [ ]:
stopwords = set(stopwords.words('english'))

words appearing maximum times in body

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords)
docs = vectorizer.fit_transform(df['body'])
features = vectorizer.get_feature_names()
visualizer = FreqDistVisualizer(features=features)
visualizer.fit(docs)
visualizer.poof()

thus India, Indian and economy government occurs many times 

for title 

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords)
docs = vectorizer.fit_transform(df['title'])
features = vectorizer.get_feature_names()
visualizer = FreqDistVisualizer(features=features)
visualizer.fit(docs)
visualizer.poof()

in title along with india corona virus, modi, pm , science occurs the most

in comments

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords)
docs = vectorizer.fit_transform(df['comments'])
features = vectorizer.get_feature_names()
visualizer = FreqDistVisualizer(features=features)
visualizer.fit(docs)
visualizer.poof()

there is much about religion in comments might be because of people in India are most inclined to religion etc

combined features

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords)
docs = vectorizer.fit_transform(df['combined_features'])
features = vectorizer.get_feature_names()
visualizer = FreqDistVisualizer(features=features)
visualizer.fit(docs)
visualizer.poof()

in combined features as well india and religion are predominant